In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
from pandas import DataFrame as df
from astropy.table import Table, vstack, Column
import os
import pdb
import numpy as np
import warnings
import resource
import timeit
start_time = timeit.default_timer()
# code you want to evaluate
resource.setrlimit(resource.RLIMIT_NOFILE, (10000,-1))

In [38]:
path = './../spectra/'
path2 = './../catalogs/'

bigFile = Table.read('bigfile.fits')
vccfiles = []
vdgcfiles = []
vugcfiles = []
for filename in os.listdir(path):
    if ((filename.endswith(".fits")) & ("spec" in filename)):
       vdgcfiles.append(filename)
    elif ((filename.endswith(".gz")) & ("spec" in filename)):
       vugcfiles.append(filename)
    elif ((filename.endswith(".fits")) & ("VCC" in filename)):
        vccfiles.append(filename)

In [69]:
stars = [] #Star 
ICGC = [] #ICL
M87 = [] #Halo
DEGC = [] #DEGC
UDG = [] #UDG
DENUC = [] #DENU
objType = []
objFile = []

In [71]:
#instead get the files then do doppler shift stuff
#restFrame unecessary, just need to get the files and velocity values
restFrame = []
VUGC = []
velocity1 = []3
for string in vugcfiles:
    file = fits.open(path + string)
    wavelength = file[1].data['LAMBDA']
    file.close()
    for otherFile in bigFile:
        if (string == otherFile['SPEC1DNAME']):
            velocityVal = otherFile['VOBS']
            VUGC.append(string)
            objType.append(otherFile['Type'])
            objFile.append(otherFile['SPEC1DNAME'])
            velocity1.append(velocityVal)
            a = (wavelength * ((1)/(1+(velocityVal/(3e+5)))))
            restFrame.append(a)
            
print(restFrame)

[array([[4675.453 , 4676.033 , 4676.6123, ..., 7215.78  , 7216.418 ,
        7217.0566]], dtype=float32), array([[4893.7886, 4894.3945, 4895.    , ..., 7457.316 , 7457.9565,
        7458.597 ]], dtype=float32), array([[4854.627 , 4855.233 , 4855.839 , ..., 7417.7124, 7418.353 ,
        7418.9937]], dtype=float32), array([[5215.943 , 5216.5513, 5217.1597, ..., 7789.429 , 7790.073 ,
        7790.717 ]], dtype=float32), array([[5133.102 , 5133.7114, 5134.3203, ..., 7698.124 , 7698.7646,
        7699.4053]], dtype=float32), array([[4868.082 , 4868.6885, 4869.2944, ..., 7432.0503, 7432.691 ,
        7433.3315]], dtype=float32), array([[4954.521 , 4955.1274, 4955.7344, ..., 7503.751 , 7504.387 ,
        7505.0244]], dtype=float32), array([[4674.832, 4675.45 , 4676.069, ..., 7225.909, 7226.544, 7227.179]],
      dtype=float32), array([[4683.4336, 4684.037 , 4684.64  , ..., 7239.979 , 7240.618 ,
        7241.258 ]], dtype=float32), array([[4510.172 , 4510.759 , 4511.3467, ..., 7058.862 , 7059.

In [4]:
print(restFrame[0][0])

[4675.453  4676.033  4676.6123 ... 7215.78   7216.418  7217.0566]


In [72]:
#instead get the files then do doppler shift stuff
#restFrame unecessary, just need to get the files and velocity values
restFrame1 = []
VDGC = []
velocity = []
for string in vdgcfiles:
    file = fits.open(path + string)      
    wavelength = file[1].data['LAMBDA']
    file.close()
    for otherFile in bigFile:
        s = otherFile['SPEC1DNAME']
        if (string == s[:-3]):
            VDGC.append(string)
            objType.append(otherFile['Type'])
            objFile.append(otherFile['SPEC1DNAME'])
            velocityVal = otherFile['VOBS']
            velocity.append(velocityVal)
            a = (wavelength * ((1)/(1+(velocityVal/(3e+5)))))
            restFrame1.append(a)

In [73]:
restFrame2 = []
VCC = []
velocity2 = []
for string in vccfiles:
    file = fits.open(path + string)
    wavelength = file[1].data['LAMBDA']
    file.close()
    for otherFile in bigFile:
        s = otherFile['VCC']
        if (string[:-5] == s):
            VCC.append(string)
            objType.append(otherFile['Type'])
            objFile.append(otherFile['VCC'])
            velocityVal = otherFile['VOBS']
            velocity2.append(velocityVal)
            a = (wavelength * ((1)/(1+(velocityVal/(3e+5)))))
            restFrame2.append(a)

In [77]:
for i in range (0, len(objType)):
    if (objType[i] == 'Star'):
        stars.append(objFile[i])
    elif (objType[i] == 'ICL'):
        ICGC.append(objFile[i])
    elif (objType[i] == 'Halo'):
        M87.append(objFile[i])
    elif (objType[i] == 'DEGC'):
        DEGC.append(objFile[i])
    elif (objType[i] == 'UDG'):
        UDG.append(objFile[i])
    elif (objType[i] == 'DENU'):
        DENUC.append(objFile[i])


In [6]:
warnings.simplefilter("ignore", RuntimeWarning)
def spec_interp(wv,fx,nwwv,*args):
    #Required arguments:
    #   - wv: old wavelength array
    #   - fx: flux to be rebinned, 'SPEC'
    #   - nwwv: new wavelen gth array, interval
    #
    #Optional argument: variance
    npix = len(wv)
    if len(args) == 0:
        var = np.ones(npix)
        nwvarFlag = False
    else:
        var = args[0]
        nwvarFlag = True
    nwpix = len(nwwv)
    #Calculate wavelength endpoints for each pixel
    wvl = (wv + np.roll(wv,1))/2.
    wvh = (wv + np.roll(wv,-1))/2.
    wvl[0] = wv[0] - (wv[1] - wv[0])/2.
    wvh[npix-1] = wv[npix-1] + (wv[npix-1]-wv[npix-2])/2.
    #Calculate endpoints of the final array
    bwv = np.zeros(nwpix+1)
    bwv[0:nwpix] = (nwwv+np.roll(nwwv,1))/2.
    bwv[0] = nwwv[0] - (nwwv[1] - nwwv[0])/2.
    bwv[nwpix] = nwwv[nwpix-1]+(nwwv[nwpix-1] - nwwv[nwpix - 1])/2.
    #Create tmp arrays for final array
    nwfx = np.zeros(nwpix)
    nwvar = np.zeros(nwpix)
    nwunitfx = np.zeros(nwpix)
    #Loop through the arrays
    for q in range(npix):
        #No overlap
        if (wvh[q] <= bwv[0]) | (wvl[q] >= bwv[nwpix]):
            continue
        #Find pixel that bw is within
        if wvl[q] <= bwv[0]:
            i1 = [0]
        else:
            i1 = np.argwhere((wvl[q] <= np.roll(bwv,-1)) & (wvl[q] > bwv))[0]
        if wvh[q] > bwv[nwpix]:
            i2 = [nwpix-1]
        else:
            i2 = np.argwhere((wvh[q] <= np.roll(bwv,-1)) & (wvh[q] > bwv))[0]
        j1 = i1[0]
        j2 = i2[0]
        #Now Sum up
        for kk in range(j1,j2+1):
            #Rejected pixesl do not get added in
            if var[q] > 0.:
                frac = ( np.min([wvh[q],bwv[kk+1]]) - np.max([wvl[q],bwv[kk]]) ) / (wvh[q]-wvl[q])
                nwfx[kk] = nwfx[kk]+frac*fx[q]
                nwunitfx[kk] = nwunitfx[kk]+frac*1.0
                #Variance
                if nwvarFlag:
                    if (var[q] <= 0.) | (nwvar[kk] == -1):
                       nwvar[kk] = -1
                    else:
                       nwvar[kk] = nwvar[kk]+frac*var[q]
    if nwvarFlag:
        fxOut = nwfx/nwunitfx
        varOut = nwvar*nwunitfx
        
        return fxOut,varOut
    else:
        fxOut = nwfx/nwunitfx
        return fxOut


def rebinspec(*args,**kwargs):
    #Required arguments:
    #   - wv: old wavelength array
    #   - fx: flux to be rebinned
    #   - nwwv: new wavelength array
    #
    #Optional arguments:
    #   - var = var, input and output variance
    #   - ivar = ivar, input and output ivar

    if len(args) != 3:
        print('Proper syntax is: out = rebinspec(wv, fx, nwwv, **kwargs)')
        return np.nan

    else:
        wv, fx, nwwv = args

        var = kwargs.get('var',None)
        ivar = kwargs.get('ivar',None)

        if (var is not None) & (ivar is None):
            nwfx,nwvar = spec_interp(wv,fx,nwwv,var)

            return nwfx, nwvar
        elif (var is None) & (ivar is not None):
            var = 1./ivar
            nwfx,nwvar_1 = spec_interp(wv,fx,nwwv,var)
            nwvar_1[nwvar_1 == 0.0] = -10.0
            nwivar = 1.0/nwvar_1
            nwivar[nwivar < 0.0] = 0.0
            
            return nwfx, nwivar
        else:
            nwfx = spec_interp(wv,fx,nwwv)

            return nwfx

In [7]:
def plotting (fileName, title, vel, colors):
    correct = np.arange(4000,8750.9,0.9)
    allFluxes = []
    allIvar = []
    allWaves = []
    i = 0
    for f in fileName: 
        file = fits.open(path + f)
        try:                                  #check for Bxspf blue or red, if not I resort to Horne
            tempB = file['Bxspf-' + 'B']
            tempR = file['Bxspf-' + 'R']
        except:
            tempB = file['Horne-' + 'B']
            tempR = file['Horne-' + 'R']
        
        allWaves.append(np.append(((tempB.data['LAMBDA'])/(1 + ((vel[i]/300000)))), ((tempR.data['LAMBDA'])/(1 + ((vel[i]/300000))))))
        allFluxes.append(np.append(tempB.data['SPEC'], tempR.data['SPEC']))
        allIvar.append(np.append(tempB.data['IVAR'], tempR.data['IVAR']))              
        i = i + 1
        file.close()
    fileFlux = np.zeros((len(allWaves), len(correct)))
    fileVar = np.zeros((len(allWaves), len(correct)))
    for i in range (0, len(allWaves)):
        newFlux, newVar = np.nan_to_num(rebinspec(allWaves[i], allFluxes[i], correct, ivar = allIvar[i]))
        '''plt.plot(correct, newFlux, color = colors)
        plt.xlabel('Wavelength')
        plt.ylabel('Flux')
        plt.title(title)
        plt.show()'''
        fileFlux[i][:] = newFlux
        fileVar[i][:] = newVar
        
    
    return fileFlux, fileVar

In [ ]:
fluxStar, varStar = plotting(VDGC, 'DE Spectra', velocity, 'mediumspringgreen')

In [8]:
flux, var = plotting(VDGC, 'DE Spectra', velocity, 'mediumspringgreen')


In [9]:
flux1, var1 = plotting(VUGC, 'UDG Spectra', velocity, 'mediumspringgreen')


KeyboardInterrupt: 

In [ ]:
flux2, var2 = plotting(VCC, 'VCC Spectra', velocity, 'mediumspringgreen')


In [10]:
def sigmaclipping(flux):
    correct = np.arange(4000,8750.9,0.9)
    ones = np.ones(np.shape(flux))
    means = np.mean(flux, axis = 0)
    stds = np.std(flux, axis = 0)
    ones[np.abs(flux - means) > 3 * stds] = 0
    a = ones * flux
    return ones

In [ ]:
# sigmaClipping(flux)
'''
correct = np.arange(4000,8750.9,0.9)
for i in range (0, 10):
    plt.plot(correct, a[i], color = 'g')
    plt.xlabel('Wavelength')
    plt.ylabel('Flux')
    plt.title('Plot')
    plt.show()'''

In [11]:
#region 6500 - 6600
# divide everything in that region of array a by that median
def normalize (flux):
    start = 2777
    end = 2889
    region = []
    np.array(region)
    medians = []
    increment = 0
    for j in range (0, len(flux)):
        for i in range (start, end):
            region = []
            region.append(flux[j][i])
            #fix this
        medians.append(np.median(region))
    for i in range (0, len(medians)):
            flux[i] = (flux[i] / medians[i])
    return flux


In [35]:
fluxVUGC = normalize(flux)
#fluxVDGC = normalize(flux1)
#fluxVCC = normalize(flux2)

In [20]:
correct = np.arange(4000,8750.9,0.9)
'''plt.figure()
for i in range (0, 10):
    plt.plot(correct, a[i])
    plt.xlabel('Wavelength')
    plt.ylabel('Flux')
    plt.title('Plot')'''

"plt.figure()\nfor i in range (0, 10):\n    plt.plot(correct, a[i])\n    plt.xlabel('Wavelength')\n    plt.ylabel('Flux')\n    plt.title('Plot')"

In [13]:
def coadd(spectra, ivar, sigma):    
    spectra = np.nan_to_num(spectra)
    ivar = np.nan_to_num(ivar)

    clip = sigmaclipping(spectra)
    coadd_spectra = np.nan_to_num(np.sum(spectra * ivar * clip, axis = 0) / np.sum(ivar * clip, axis = 0))
    coadd_ivar = np.nan_to_num(np.sum(ivar * clip, axis = 0))
    coadd_err = np.nan_to_num(1 / np.sqrt(np.sum(ivar * clip, axis = 0)))

    return coadd_spectra, coadd_ivar, coadd_err

In [15]:
coaddSpec, coaddIvar, coaddErr = coadd(fluxVUGC, var, 3)